In [1]:
import awswrangler as wr

In [27]:
wr.catalog.create_parquet_table(
    database='zipcode',
    table='zip_code_governed_table',
    path='s3://aaa-demo-test-two/zipcode/',
    columns_types={'zip_code': 'int', 
                   'latitude': 'string',
                   'longitude': 'string', 
                   'city': 'string',
                  'state':'string',
                  'country': 'string',},
    compression= 'snappy',
    description= 'zip code information table',
    columns_comments= {'zip_code': 'zip_code id ', 
                      'latitude': 'latitude of country', 
                      'longitude': 'longitude of country',
                      'city': 'city name',
                      'state': 'state name',
                      'country': 'countyr name'},
    
    table_type = "GOVERNED"
    )

In [28]:
df = wr.s3.read_csv(r's3://aaa-demo-test-two/zipcode/')
df.head()

,zip_code,latitude,longitude,city,state,country
0,501,40.922326,-72.637078,Holtsville,NY,Suffolk
1,544,40.922326,-72.637078,Holtsville,NY,Suffolk
2,601,18.165273,-66.722583,Adjuntas,PR,Adjuntas
3,602,18.393103,-67.180953,Aguada,PR,Aguada
4,603,18.455913,-67.145780,Aguadilla,PR,Aguadilla


In [29]:
filtered_df = df[df.state.isin(['NY'])]
filtered_df.head()

,zip_code,latitude,longitude,city,state,country
0,501,40.922326,-72.637078,Holtsville,NY,Suffolk
1,544,40.922326,-72.637078,Holtsville,NY,Suffolk
2297,6390,40.992288,-72.723496,Fishers Island,NY,Suffolk
3259,10001,40.750422,-73.996328,New York,NY,New York
3260,10002,40.718758,-73.986427,New York,NY,New York


#  Write data to s3

In [30]:
transaction_id = wr.lakeformation.start_transaction(read_only=False)
transaction_id

'a52037ec2dac4a2dbbeafa47a34ec6f1'

In [31]:
wr.s3.to_parquet(
    df=filtered_df,
    path=f's3://aaa-demo-test-two/zipcode/',
    dataset=True,
    compression= 'snappy',
    database='zipcode',
    table='zip_code_governed_table',
    parameters={"num_cols": str(len(df.columns)), "num_rows": str(len(df.index))},
    mode='overwrite',
    table_type="GOVERNED",
    transaction_id = transaction_id
)


{'paths': ['s3://aaa-demo-test-two/zipcode/1eac458811304137ab4e43994baf1c8c.snappy.parquet'],
 'partitions_values': {}}

In [32]:
wr.lakeformation.commit_transaction(transaction_id)
transaction = wr.lakeformation.describe_transaction(transaction_id)
transaction

'committed'

In [33]:
filtered_df2 = df[df.state.isin(['NY'])]

In [34]:
transaction_id2 = wr.lakeformation.start_transaction(read_only=False)
transaction_id2

'a72fbf7e80304646a64ceb57412e90b6'

In [35]:
wr.s3.to_parquet(
    df=filtered_df,
    path=f's3://aaa-demo-test-two/zipcode/',
    dataset=True,
    compression= 'snappy',
    database='zipcode',
    table='zip_code_governed_table',
    parameters={"num_cols": str(len(df.columns)), "num_rows": str(len(df.index))},
    mode='append',
    table_type="GOVERNED",
    transaction_id = transaction_id2
)
wr.lakeformation.commit_transaction(transaction_id2)

C:\Users\opera\anaconda3\lib\site-packages\awswrangler\_data_types.py:719: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].astype(desired_type)


In [37]:
df3 = wr.lakeformation.read_sql_query(sql=f"SELECT * FROM {'zip_code_governed_table'};",
      database = 'zipcode')
df3.head()
df.shape[0]

42049

In [38]:
df4 = wr.lakeformation.read_sql_query(sql=f"SELECT * FROM {'zip_code_governed_table'};",
      database = 'zipcode',
      transaction_id = transaction_id)
df4.shape[0]

2232